In [148]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

In [149]:
train_loader = torch.load("../data/train_loader.pt")
val_loader = torch.load("../data/val_loader.pt")
test_loader = torch.load("../data/test_loader.pt")

In [150]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.act3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2)
        self.conv4 = nn.Conv2d(8, 4, kernel_size=3, padding=1)
        self.act4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(4 * 14 * 14, 32)
        self.act5 = nn.ReLU()
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = self.pool3(self.act3(self.conv3(out)))
        out = self.pool4(self.act4(self.conv4(out)))
        out = out.view(-1, 4 * 14 * 14)
        out = self.act5(self.fc1(out))
        out = self.fc2(out)
        
        return out

In [151]:
device = torch.device("cuda" if torch.cuda.is_available() else torch.device("cpu"))

In [152]:
def training_loop(n_epochs, model, criterion, optimizer, train_loader, val_loader):
    train_losses = []
    val_losses = []
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
            
        avg_loss_train = loss_train / len(train_loader)
        train_losses.append(avg_loss_train)
        
        if val_loader is not None:
            loss_val = 0.0
            with torch.no_grad():
                for val_images, val_labels in val_loader:
                    val_images, val_labels = val_images.to(device), val_labels.to(device)
                    val_outputs = model(val_images)
                    loss_val += criterion(val_outputs, val_labels).item()
            avg_val_loss = loss_val / len(val_loader)
            val_losses.append(avg_val_loss)

        print('{} Epoch {}, Training loss: {:.4f}'.format(
            datetime.datetime.now(), epoch, avg_loss_train))
        if val_loader is not None:
            print('{} Epoch {}, Validation loss: {:.4f}'.format(
                datetime.datetime.now(), epoch, avg_val_loss))

    return train_losses, val_losses

In [153]:
def plot_losses(train_losses, val_losses):
    plt.plot(train_losses, label='Training loss')
    plt.plot(val_losses, label='Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss Curve')
    plt.show()

In [154]:
n_epochs = 20
model = Net()
batch_size = 64
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)

In [155]:
train_losses, val_losses = training_loop(n_epochs, model, criterion, optimizer, train_loader, val_loader)

2024-04-11 22:21:15.216089 Epoch 1, Training loss: 0.4605
2024-04-11 22:21:15.216328 Epoch 1, Validation loss: 0.4280
2024-04-11 22:23:55.732740 Epoch 2, Training loss: 0.3711
2024-04-11 22:23:55.733006 Epoch 2, Validation loss: 0.3823
2024-04-11 22:26:38.386686 Epoch 3, Training loss: 0.3425
2024-04-11 22:26:38.386927 Epoch 3, Validation loss: 0.3496
2024-04-11 22:29:21.611459 Epoch 4, Training loss: 0.3298
2024-04-11 22:29:21.611592 Epoch 4, Validation loss: 0.3291
2024-04-11 22:32:05.748610 Epoch 5, Training loss: 0.3184
2024-04-11 22:32:05.748700 Epoch 5, Validation loss: 0.3439
2024-04-11 22:34:50.410178 Epoch 6, Training loss: 0.3121
2024-04-11 22:34:50.410495 Epoch 6, Validation loss: 0.3265
2024-04-11 22:37:33.357388 Epoch 7, Training loss: 0.2959
2024-04-11 22:37:33.357579 Epoch 7, Validation loss: 0.3613
2024-04-11 22:40:17.253459 Epoch 8, Training loss: 0.2883
2024-04-11 22:40:17.253852 Epoch 8, Validation loss: 0.3649
2024-04-11 22:43:00.702532 Epoch 9, Training loss: 0.279

KeyboardInterrupt: 

In [ ]:
plot_losses(train_losses, val_losses)

In [ ]:
val_loss = 0.0
correct = 0
total = 0

model.eval()
with torch.no_grad():
    for val_images, val_labels in val_loader:
        val_images, val_labels = val_images.to(device), val_labels.to(device)
        val_outputs = model(val_images)
        val_loss += criterion(val_outputs, val_labels).item()
        _, predicted = val_outputs.max(1)
        total += val_labels.size(0)
        correct += (predicted == val_labels).sum().item()

print('Validation loss: {:.4f}'.format(val_loss / len(val_loader)))
print('Validation accuracy: {:.2f}%'.format(100 * correct / total))

In [ ]:
test_loss = 0.0
correct = 0
total = 0

model.eval()
with torch.no_grad():
    for t_images, t_labels in test_loader:
        t_images, t_labels = t_images.to(device), t_labels.to(device)
        t_outputs = model(t_images)
        test_loss += criterion(t_outputs, t_labels).item()
        _, predicted = t_outputs.max(1)
        total += t_labels.size(0)
        correct += (predicted == t_labels).sum().item()

print('Test loss: {:.4f}'.format(test_loss / len(test_loader)))
print('Test accuracy: {:.2f}%'.format(100 * correct / total))

In [ ]:
model.eval()
predictions = []
test_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        predictions.extend(outputs.tolist())
        test_labels.extend(labels.tolist())

probs = torch.tensor(predictions)
predicted_classes = torch.argmax(probs, dim=1)

In [ ]:
print(classification_report(predicted_classes, test_labels))

In [ ]:
conf_matrix = confusion_matrix(test_labels, predicted_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix')
plt.show()